In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
sns.set(style="white", color_codes=True)
sns.set_context(rc={'font.family': 'sans', 'font.size': 24, 'axes.titlesize':24, 'axes.labelsize':24})
#if you want to know current working dir
os.chdir('../..')

from src.utils import *
from src.utility import merge_speed_events
import src.data as data
import src.utility as utils
from src.preprocessing.other_features import avg_speed_for_roadtype_event

%matplotlib inline

In [2]:
min_datetime = data.speeds_original(mode='train').DATETIME_UTC.min()
max_datetime = data.speeds_original(mode='test').DATETIME_UTC.max()
print(min_datetime, max_datetime)

caching resources/dataset/originals/speeds_train.csv.gz
caching resources/dataset/originals/speeds_test.csv.gz
2018-09-01 00:00:00 2019-01-01 23:45:00


In [3]:
df = pd.DataFrame(pd.date_range(min_datetime, max_datetime, freq='15min').to_series()).reset_index()
df[DATETIME] = pd.to_datetime(df['index'])
df = df[[DATETIME]]
df.head(10)

,DATETIME_UTC
0,2018-09-01 00:00:00
1,2018-09-01 00:15:00
2,2018-09-01 00:30:00
3,2018-09-01 00:45:00
4,2018-09-01 01:00:00
5,2018-09-01 01:15:00
6,2018-09-01 01:30:00
7,2018-09-01 01:45:00
8,2018-09-01 02:00:00
9,2018-09-01 02:15:00


In [4]:
df['WEEK_DAY'] = pd.to_datetime(df[DATETIME]).dt.weekday
df['IS_WEEKEND'] = df.WEEK_DAY.map(lambda x: x in [5, 6])
df.head(30)

,DATETIME_UTC,WEEK_DAY,IS_WEEKEND
0,2018-09-01 00:00:00,5,True
1,2018-09-01 00:15:00,5,True
2,2018-09-01 00:30:00,5,True
3,2018-09-01 00:45:00,5,True
4,2018-09-01 01:00:00,5,True
5,2018-09-01 01:15:00,5,True
6,2018-09-01 01:30:00,5,True
7,2018-09-01 01:45:00,5,True
8,2018-09-01 02:00:00,5,True
9,2018-09-01 02:15:00,5,True
